In [3]:
# Imports
import dask.distributed
import dask.diagnostics

import folium
import geopandas as gpd

import shapely.geometry
import pyproj

from IPython.display import display
from pystac_client import Client

# the data cube bit
from odc.stac import configure_rio, stac_load 

In [32]:
#common variables
lon, lat = -1.3144, 51.5755 # Atlas building at RAL

EURL = "https://earth-search.aws.element84.com/v1"
CURL = "https://api.stac.ceda.ac.uk/"

ecoll = "sentinel-2-l2a"
ccoll = "sentinel2_ard"

In [46]:
# Set up a bounding box to limit te search
km2deg = 1.0 / 111
x, y = (lon, lat)  # Center point of a query
r = 25 * km2deg
bbox = (x - r, y - r, x + r, y + r)

# SEARCH
catalog = Client.open(CURL)

query = catalog.search(
    collections=[ccoll], datetime="2023-06-15", limit=100, bbox=bbox
)

items = list(query.items())
print(f"Found: {len(items):d} datasets")

# Convert STAC items into a GeoJSON FeatureCollection
stac_json = query.item_collection_as_dict()

Found: 4 datasets


In [ ]:
# def xarr_from_aoi_point(point: Point, time_range: str = "2023-01-01/2024-01-01", max_cloud_cover: int = 10, max_nodata_percent: int = .5) -> xr.DataArray:
#     collection_id = "sentinel-2-c1-l2a"
#     catalog = pystac_client.Client.open(
#     "https://earth-search.aws.element84.com/v1",
#     )

#     def perform_search(tile: str = None):
#         query_params = {"eo:cloud_cover": {"lt": max_cloud_cover}, "s2:nodata_pixel_percentage": {"lt":max_nodata_percent}}
#         if tile:
#             query_params['grid:code'] = {'eq': tile}
        
#         search = catalog.search(collections=[collection_id], 
#                                 intersects=point, 
#                                 datetime=time_range, 
#                                 query=query_params, 
#                                 max_items=3,
#                                 sortby="properties.datetime")
#         return search.item_collection()
    
#     items = perform_search()
    if len(set([item.properties['grid:code'] for item in items])) != 1:
        tile_code = next((item.properties.get('grid:code') for item in items if 'grid:code' in item.properties), None)
        items = perform_search(tile=tile_code)
    xarr = stac_load(
        items,
        bands=("visual.1", "visual.2", "visual.3"),
        chunks={},  # <-- use Dask
        crs = items[0].properties['proj:epsg']
    )
    return items


In [47]:
a = query.item_collection()

In [48]:
b = a.items[1]
c = b.to_dict()
c

{'type': 'Feature',
 'stac_version': '1.0.0',
 'id': 'neodc.sentinel_ard.data.sentinel_2.2023.06.15.S2B_20230615_latn518lonw0008_T30UXC_ORB137_20230615131239_utm30n_osgb',
 'properties': {'file_count': 8,
  'start_datetime': '2023-06-15T11:06:29Z',
  'end_datetime': '2023-06-15T11:06:29Z',
  'NSSDC Identifier': '2015-000A',
  'created': '2024-02-07T12:20:32.968742Z',
  'Instrument Family Name': 'Multi-Spectral Instrument',
  'Platform Number': '2B',
  'Datatake Type': 'INS-NOBS',
  'esa_file_name': 'S2B_MSIL1C_20230615T110629_N0509_R137_T30UXC_20230615T131239',
  'Ground Tracking Direction': 'ascending',
  'datetime': '2023-06-15T11:06:29Z',
  'instance_id': 'neodc.sentinel_ard.data.sentinel_2.2023.06.15.S2B_20230615_latn518lonw0008_T30UXC_ORB137_20230615131239_utm30n_osgb',
  'size': 2096421365,
  'Product Type': 'S2MSI1C',
  'Instrument Family Name Abbreviation': 'MSI',
  'Start Orbit Number': '032767',
  'eo:cloud_cover': '49.1430917614739',
  'Start Relative Orbit Number': '137',
 